In [9]:
from recordtype import recordtype
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
Task = recordtype('Task','weight duration due done')

In [14]:
def create_tasks(num):
    tasks = []
    dist = np.round(np.random.exponential(5,10000))
    for i in range(num):
        due = np.random.choice(dist)+1
        weight = np.random.randint(1,100)
        duration = np.random.randint(0,due)
#         print(due,weight,duration)
        t = Task.append(weight=weight, duration = duration, due=due,done=0)

In [15]:
def tally(comp_task,all_task):
    completed = len(comp_task)/len(all_task)
    
#     we set 75th percentile of all tasks based on weight as important tasks
    percentile = int(0.75*len(all_task))
    imp_tasks = sorted(all_task, key = lambda x: x.weight)[percentile:len(all_task)]
    
#     check how many important tasks are completed
    overlap = 0
    for i in imp_tasks:
        if i in comp_task:
            overlap += 1
    if len(imp_tasks) > 0:
        important = overlap/len(imp_tasks)
    else:
        important = 0
    
    intime = []
    for i in comp_task:
        if i.due >= i.done:
            intime.append(i)
    intime_task = len(intime)/len(all_task)
    
    return (completed,important,intime_task)

In [16]:
def duration(t):
    return t.duration

def simulate(ratio,*tasks):
    deadline = ratio *sum(list(map(duration,tasks)))
    completed_tasks = []
    elapsed_time = 0
    for task in tasks:
        elapsed_time += task.duration
        if elapsed_time <= deadline:
            task.done = elasped_time
            completed_tasks.append(task)
        else:
            break
    return tally(completed_tasks,tasks)